# 03 SDK | 05 Chat Tools

## Tool Support

Chat tools can work in conjunction with a large language model (LLM) such as GPT-4 by allowing developers to provide a set of function definitions to the model which helps the LLM to complete chat completion requests. 

### Execution Steps

![Overview](../../media/img/03_SDK/05_ChatTools_Overview.png)

- **(1) Create ChatCompletion Request:** A default ChatCompletion request is created using a default system and user message. Additionally, `ChatCompletionsFunctionToolDefinition` objects can be provided, which contain function descriptions, parameters and return values. The LLM can then check if any of the provided functions can help fulfill the chat completion request and return the selected functions and parameters needed to call the function to the caller.
- **(2) Execute Function:** The caller can then execute the selected functions in its own security context and with credentials he owns.
- **(3) Provide Function Call Results:** Results from the function calls are provided to the LLM together with the chat history to fulfill the initial request. The LLM takes the additional input data to fulfill the request.


### Further information

- [MS Learn - How to use function calling with Azure OpenAI](https://learn.microsoft.com/en-us/azure/ai-services/openai/how-to/function-calling)
- [OpenAI - Function calling](https://platform.openai.com/docs/guides/function-calling)
  

## Azure Environment

To execute the sample code Azure service specific information like endpoint, api key etc. is needed. ([Details and instructions can be found here](../01_DemoEnvironment/01_Environment.ipynb)). Function calling is currently in preview and is supported by the latest GPT-35-Turbo and GPT-4 models.

## Step 1: Create OpenAIClient

The OpenAIClient from Azure.AI.OpenAI is a .NET client library that acts as the centralized point for all .NET functionality that want to interact with a deployed Azure OpenAI Large Language Model. It provides methods to access the OpenAI REST APIs for various tasks such as text completion, text embedding, and chat completion, etc.. It also allows developers to specify the model, engine, and options for each request, such as temperature, frequency penalty, presence penalty, and stop sequences. 

The OpenAIClient can connect to any Azure OpenAI resource or to the non-Azure OpenAI inference endpoint, making it a versatile and powerful tool for .NET development with OpenAI.


In [1]:
#r "nuget: Azure.AI.OpenAI, 1.0.0-beta.12"
#r "nuget: DotNetEnv, 2.5.0"

using Azure; 
using Azure.AI.OpenAI;
using DotNetEnv;
using System.IO;
using System.Text.Json; 

//configuration file is created during environment creation
//if you skipped the deployment just remove the code and provide values from your deployment
static string _configurationFile = @"../01_DemoEnvironment/conf/application.env";
Env.Load(_configurationFile);

string oAiApiKey = Environment.GetEnvironmentVariable("SKIT_AOAI1106_APIKEY") ?? "SKIT_AOAI1106_APIKEY not found";
string oAiEndpoint = Environment.GetEnvironmentVariable("SKIT_AOAI1106_ENDPOINT") ?? "SKIT_AOAI1106_ENDPOINT not found";
string chatCompletionDeploymentName = Environment.GetEnvironmentVariable("SKIT_AOAI1106_DEPLOYMENTNAME") ?? "SKIT_AOAI1106_DEPLOYMENTNAME not found";
string assetsFolder = Path.Combine(Directory.GetCurrentDirectory(), "..", "..", "assets");

AzureKeyCredential azureKeyCredential = new AzureKeyCredential(oAiApiKey);
OpenAIClient openAIClient = new OpenAIClient(new Uri(oAiEndpoint), azureKeyCredential);


Console.WriteLine($"OpenAI Client created...");

Installed Packages Azure.AI.OpenAI, 1.0.0-beta.12 DotNetEnv, 2.5.0

OpenAI Client created...


Expected output:

```
Installed Packages
    Azure.AI.OpenAI, 1.0.0-beta.12
    DotNetEnv, 2.5.0

OpenAI Client created...
```

## Step 2: Create Tool Definitions

The `ChatCompletionsFunctionToolDefinition` class can be used to describe locally available functions that can help the LLM to retrieve additional information to fulfill a requested. This class provides information such as the function description, parameters, and return values, which are used to explain the function and its behavior to the LLM.

In this example two 'tools' are defined:
- `toolUserDetails`: can be leveraged to retrieve additional information about a specific user.
- `toolOrderDetails`: can be leveraged to retrieve expected delivery dates of orders.

In [2]:
//Define tool(s)
ChatCompletionsFunctionToolDefinition toolUserDetails = new ChatCompletionsFunctionToolDefinition() {
    Name = "SKit_Customer_GetCustomerDetails",
    Description = "Get customer details. Function returns all kind of information of registered customers this include first name and sure name",
    Parameters = BinaryData.FromObjectAsJson(
        new {
            Type = "object",
            Properties = new {
                Id = new {
                    Type = "string",
                    Description = "The customer Id (i.e. DK3-008AF)"
                }
            },
            Required = new[] { 
                "Id" 
            }
        },
        new JsonSerializerOptions() { PropertyNamingPolicy = JsonNamingPolicy.CamelCase }
    )   
}; 

ChatCompletionsFunctionToolDefinition toolOrderDetails = new ChatCompletionsFunctionToolDefinition() {
    Name = "SKit_Order_GetOrderDeliveryDate",
    Description = "Provides the expected delivery data of an order.",
    Parameters = BinaryData.FromObjectAsJson(
        new {
            Type = "object",
            Properties = new {
                OrderId = new {
                    Type = "string",
                    Description = "The ID of the order (i.e. 4711)"
                },
                CustomerId = new {
                    Type = "string",
                    Description = "The ID or name of the person who has ordered goods"
                }
            },
            Required = new[] { 
                "OrderId" 
            }
        },
        new JsonSerializerOptions() { PropertyNamingPolicy = JsonNamingPolicy.CamelCase }
    )   
}; 

Console.WriteLine("Tool: SDK_Order_GetOrderDeliveryDate() defined");
Console.WriteLine("Tool: SDK_Customer_GetCustomerDetails() defined");


Tool: SDK_Order_GetOrderDeliveryDate() defined
Tool: SDK_Customer_GetCustomerDetails() defined


Expected output: 
```
Tool: SDK_Order_GetOrderDeliveryDate() defined
Tool: SDK_Customer_GetCustomerDetails() defined
```

## Step 3: Compose ChatCompletionsOptions

Each chat would follow similar structure, where _System_, _Agent_ and _User_ messages are added in sequence. Parameters, such as _Temperature_ could be set per call. Additionally to system message and user message a list of `ChatCompletionsFunctionToolDefinition` instances which define tool functions can be provided.

In this example the two previously defined tools `toolUserDetails` and `toolOrderDetails` are added to the `ChatCompletionsOptions` instance.


In [3]:
//Define System Prompt
string systemMessage = @"You are an AI assistant that helps customers find information about their orders. 
    There's no need to ask for permission if you need additional tooling to fulfill the information request. 
    You answer with the first name and sure name of the customer.
    You always answer with: 'We expect your order to arrive at: {expected time of arrival}. This information was created for: {first name} {last name}'";

string userMessage = @"Hey, here's John.
    I'm calling on behalf of Contoso and I need information for regarding my order number 4711.
    The order was placed on March 1st where customer number is 984-AB489. Can you tell me when it will be delivered?
";

//Define chat completion options
var chatCompletionsOptions = new ChatCompletionsOptions()
{
    DeploymentName = chatCompletionDeploymentName,
    Messages = { 
        new ChatRequestSystemMessage(systemMessage),
        new ChatRequestUserMessage(userMessage) 
    },
    ToolChoice = ChatCompletionsToolChoice.Auto,
    Tools = { 
        toolUserDetails, 
        toolOrderDetails
    },
};


Console.WriteLine($"ChatCompletionsOptions created...");



ChatCompletionsOptions created...


Expected output:

```
ChatCompletionsOptions created...
```

## Step 4: Call ChatCompletion API

The first interaction with the LLM is to call the `GetChatCompletionAsync()` method to get the response from the LLM. Based on the provided system and user message we expect that LLM to return with a request to call additional locally available functions. The LLM provides the information that functions should be called by setting `chatChoice.FinishReson == CompletionsFinishReason.ToolCalls`. Note that function name and function call parameters are provided. The code cell is just showing the functions which should be called but doesn't call them yet. This will be done in one of the following cells.

In [4]:
//LLM Call
OpenAIClient openAIClient = new OpenAIClient(new Uri(oAiEndpoint), new AzureKeyCredential(oAiApiKey));
Response<ChatCompletions> response = await openAIClient.GetChatCompletionsAsync(chatCompletionsOptions);

//Tool Function Call
Console.WriteLine($"LLM call finish reason: {response.Value.Choices[0].FinishReason}");

List<ChatRequestToolMessage> chatRequestToolMessages = new List<ChatRequestToolMessage>();
ChatChoice chatChoice = response.Value.Choices[0]; 

//List functions which should be called
if (chatChoice.FinishReason == CompletionsFinishReason.ToolCalls) {
    foreach (ChatCompletionsToolCall toolCall in chatChoice.Message.ToolCalls)
    {
        if (toolCall is ChatCompletionsFunctionToolCall functionToolCall)
        {
            if (functionToolCall != null)
            {
                Console.WriteLine($"Function call: {functionToolCall.Name} with arguments: {functionToolCall.Arguments}");
            }
        }
    }
}


LLM call finish reason: tool_calls
Function call: SKit_Customer_GetCustomerDetails with arguments: {"id": "984-AB489"}
Function call: SKit_Order_GetOrderDeliveryDate with arguments: {"orderId": "4711", "customerId": "984-AB489"}


Expected output:

```
LLM call finish reason: tool_calls
Function call: SKit_Customer_GetCustomerDetails with arguments: {"id": "984-AB489"}
Function call: SKit_Order_GetOrderDeliveryDate with arguments: {"orderId": "4711", "customerId": "984-AB489"}
```

## Step 5: Define Helper Class

In the previous step the LLM response provided information which function need to be called. This cell defines a helper function to call functions from DLLs using reflection. The library which is dynamically loaded is build using this source:

```csharp
namespace SKit; 

using System.Text.Json;

public class Order
{
    public DateTime GetOrderDeliveryDate(string orderId, string? customerId = null){
        return DateTime.Now.AddDays(5);
    }
}

public class Customer
{
    public string GetCustomerDetails(string id)
    {
        //convert to json string

        return JsonSerializer.Serialize(new {
            FirstName = "Jon",
            LastName = "Doe",
            Company = "Contoso",
            Title = "CTO"
        });
    }
}
```

Function name and parameter retrieved from the LLM are used to locate the specific function in the dll, instantiate the hosting class and finally execute it.

In [5]:
using System.IO;
using System.Reflection;
using System.Text.Json;

#nullable enable
private string CallFunctionViaReflection(string name, string arguments)
{
    name = name.Replace("_", ".");
    string typeName = name.Substring(0, name.LastIndexOf("."));
    string methodName = name.Substring(name.LastIndexOf(".") + 1);

    string dllFilePath = Path.Combine(assetsFolder, "docs", "03_SDK", "FunctionCollection.dll");
    Assembly assembly = Assembly.LoadFrom(dllFilePath);
    Type? type = assembly.GetType(typeName);

    MethodInfo? method = type?.GetMethod(methodName);
    object? instance = null;
    if (type!=null) {
        instance = Activator.CreateInstance(type);
    }

    JsonElement jsonElement = JsonDocument.Parse(arguments).RootElement;
    int i = 0;
    if (method == null) { return ""; }
    object[] methodArguments = new object[method.GetParameters().Length];
    foreach (var parameter in method.GetParameters())
    {
        if (parameter.Name == null) { continue; }
        var parameterValue = jsonElement.GetProperty(parameter.Name).ToString();
        methodArguments[i++] = parameterValue;
    }

    if (method != null && instance != null) {
        object? result = method.Invoke(instance, methodArguments);
        return Convert.ToString(result) ?? "";
    }
    
    return ""; 
}

Console.WriteLine($"Helper to call function via reflection from dll defined...");


Helper to call function via reflection from dll defined...


## Steps 6: Call Local Function(s)

The previously defined helper class performs function calls suggested by the LLM response. Results from the function calls are added as `ChatRequestToolMessage` to a list of `ChatRequestToolMessage`

In [6]:
List<ChatRequestToolMessage> chatRequestToolMessages = new List<ChatRequestToolMessage>();
if (chatChoice.FinishReason == CompletionsFinishReason.ToolCalls) {
    foreach (ChatCompletionsToolCall toolCall in chatChoice.Message.ToolCalls)
    {
        if (toolCall is ChatCompletionsFunctionToolCall functionToolCall)
        {
            if (functionToolCall != null)
            {
                string functionReturnValue = CallFunctionViaReflection(functionToolCall.Name, functionToolCall.Arguments);
                Console.WriteLine($"Function call: {functionToolCall.Name} with arguments: {functionToolCall.Arguments} returned {functionReturnValue}");
                chatRequestToolMessages.Add(
                    new ChatRequestToolMessage(
                        functionReturnValue, 
                        toolCall.Id
                    )
                );
            }
        }
    }
}

Function call: SKit_Customer_GetCustomerDetails with arguments: {"id": "984-AB489"} returned {"FirstName":"Jon","LastName":"Doe","Company":"Contoso","Title":"CTO"}
Function call: SKit_Order_GetOrderDeliveryDate with arguments: {"orderId": "4711", "customerId": "984-AB489"} returned 2/5/2024 2:01:03 PM


Expected output:

```
Function call: SKit_Customer_GetCustomerDetails with arguments: {"id": "984-AB489"} returned {"FirstName":"Jon","LastName":"Doe","Company":"Contoso","Title":"CTO"}
Function call: SKit_Order_GetOrderDeliveryDate with arguments: {"orderId": "4711", "customerId": "984-AB489"} returned 2/4/2024 6:23:22 PM
```

## Step 7: Finalize LLM Call

In the final step we provide the results from the local function calls to a new chat completion request. Note that the LLM incorporates information from both function calls in it's response. 

It provides 'Doe' even if the initial user message just mentions 'Hey, here's John' as well as the estimated delivery date of the order.

In [8]:
if (chatChoice.FinishReason == CompletionsFinishReason.ToolCalls)
{
    // Create conversation history
    ChatRequestAssistantMessage chatRequestAssistantMessage = new(chatChoice.Message);
    chatCompletionsOptions.Messages.Add(chatRequestAssistantMessage);

    // Provide function call results
    foreach (var chatRequestToolMessage in chatRequestToolMessages) {
        chatCompletionsOptions.Messages.Add(chatRequestToolMessage);
    }

    response = await openAIClient.GetChatCompletionsAsync(chatCompletionsOptions);
    Console.WriteLine(response.Value.Choices[0].Message.Content); 
}


We expect your order to arrive at: 2/5/2024 2:01:03 PM. This information was created for: Jon Doe


Expected output:

```
We expect your order to arrive at: 2/4/2024 6:23:22 PM. This information was created for: Jon Doe
```


## Next Steps

- The concept of Embeddings allows transforming information into a numerical representation preserving the semantic context of the information: [Demo Embeddings](../04_Embeddings/01_BasicEmbeddings.ipynb)